In [1]:
import cv2
import easyocr
import numpy as np
import imutils

video_path = r"C:\Users\SOURI\Downloads\V508SD Number Plate Quality Test Distance In Metres CCTV Camera Review.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

reader = easyocr.Reader(['en'])

# Get the FPS of the video
fps = cap.get(cv2.CAP_PROP_FPS)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define color range for detecting yellow (for instance)
    lower_yellow = np.array([15, 100, 100])  # Adjust based on the number plate color
    upper_yellow = np.array([35, 255, 255])
    
    # Create mask for detecting yellow regions
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    
    # Morphological transformations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)

    # Find contours based on the mask
    keypoints = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    location = None
    for contour in contours:
        # Approximate the contour to a polygon
        approx = cv2.approxPolyDP(contour, 10, True)
        
        # Check if the contour is a quadrilateral
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            
            # Relax aspect ratio condition to allow for distant plates
            if 2 < aspect_ratio < 5 and cv2.contourArea(contour) > 1000:  # Set a minimum contour area to avoid small noise
                location = approx
                break

    if location is not None:
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [location], -1, 255, -1)
        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = frame[x1:x2+1, y1:y2+1]
        cropped_image = cv2.resize(cropped_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

        # Perform OCR to detect the number plate text
        result = reader.readtext(cropped_image)
        if result:
            text = result[0][-2]
            
            # Draw text and bounding box on the frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            frame = cv2.putText(frame, text=text, org=(approx[0][0][0], approx[1][0][1] + 60),
                                fontFace=font, fontScale=1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
            frame = cv2.rectangle(frame, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3)

    # Display the frame with correct delay
    cv2.imshow("Video Processing", frame)

    # Wait for the right amount of time based on the FPS of the video
    if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
